# 3.a. Comprehensive Student Performance Analysis Using Pandas
#### Problem Statement:

Perform a thorough analysis of student performance across multiple subjects while handling data issues such as missing marks, duplicate entries, and inconsistent formatting. Use various Pandas functionalities to clean, organize, and derive insights from the data.

#### Objective Scenario:

A college stores internal assessment scores of students in an Excel/CSV file for three subjects: Math, Physics, and Chemistry. However, the dataset contains:

•	Missing marks due to absent students,
•	Duplicate entries for some students,
•	Inconsistent string formatting in names,
•	No unique ID assigned to students.

The academic head wants to:

1.	Clean and prepare the data.
2.	Analyze individual and subject-wise performance.
3.	Identify high-performing students and subjects with the most variability.

#### Dataset:

The data is stored in a CSV file named student_scores.csv, and contains the following columns:

Example data:

Name,Math,Physics,Chemistry

Alice,78,85,

Bob,82,,88

Charlie,75,79,91

Alice,78,85,

David,,92,87

Eve,85,84,88

Bob,82,,88

#### Tasks to be Performed:

🔹 Data Loading and Inspection

•	Load the dataset using Pandas from the CSV file.
•	Display basic information about the dataset using .info() and .describe().
•	Count the number of missing entries in each subject column.
•	Detect and display duplicate rows based on student names.

🔹 Data Cleaning

•	Remove duplicate entries while keeping only the first occurrence of each student.
•	Replace missing subject scores with the median of that subject using .fillna().
•	Assign a new unique identifier column StudentID starting from 1.

🔹 Data Transformation and Normalization

•	Add a new column TotalMarks which sums the marks of all three subjects for each student.
•	Create another column AverageMarks as total marks divided by the number of subjects.
•	Apply min-max normalization on all three subject scores to scale them between 0 and 1, and store them in new columns: Math_Norm, Physics_Norm, Chemistry_Norm.

🔹 Statistical and Performance Analysis

•	Calculate the mean, median, and standard deviation for each subject using normalized scores.
•	Identify the subject with the highest standard deviation (i.e., the most variable scores).
•	Identify the top 3 students based on their average marks.
•	Display subject-wise average performance using a pivot table.



In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# -----------------------------------------
# 🔹 1. Data Loading and Inspection
# -----------------------------------------
# Load dataset from CSV
df = pd.read_csv("/content/student_scores.csv")

print("\n--- Dataset Info ---")
print(df.info())

print("\n--- Dataset Description ---")
print(df.describe())

# Count missing values per subject
print("\nMissing values per subject:")
print(df[["Math", "Physics", "Chemistry"]].isna().sum())

# Detect duplicates based on Name
duplicates = df[df.duplicated(subset=["Name"], keep=False)]
print("\nDuplicate Entries:\n", duplicates)

# -----------------------------------------
# 🔹 2. Data Cleaning
# -----------------------------------------
# Remove duplicates (keep first occurrence)
df = df.drop_duplicates(subset=["Name"], keep="first")

# Replace missing scores with median of that subject
df["Math"].fillna(df["Math"].median(), inplace=True)
df["Physics"].fillna(df["Physics"].median(), inplace=True)
df["Chemistry"].fillna(df["Chemistry"].median(), inplace=True)

# Assign unique StudentID
df.insert(0, "StudentID", range(1, len(df) + 1))

print("\n--- Cleaned Data ---")
print(df)

# -----------------------------------------
# 🔹 3. Data Transformation & Normalization
# -----------------------------------------
# Total and Average Marks
df["TotalMarks"] = df[["Math", "Physics", "Chemistry"]].sum(axis=1)
df["AverageMarks"] = df["TotalMarks"] / 3

# Min-Max Normalization
scaler = MinMaxScaler()
df[["Math_Norm", "Physics_Norm", "Chemistry_Norm"]] = scaler.fit_transform(
    df[["Math", "Physics", "Chemistry"]]
)

print("\n--- Data with Normalized Scores ---")
print(df)

# -----------------------------------------
# 🔹 4. Statistical & Performance Analysis
# -----------------------------------------
# Mean, Median, Std for normalized scores
stats = {
    "Mean": df[["Math_Norm", "Physics_Norm", "Chemistry_Norm"]].mean(),
    "Median": df[["Math_Norm", "Physics_Norm", "Chemistry_Norm"]].median(),
    "StdDev": df[["Math_Norm", "Physics_Norm", "Chemistry_Norm"]].std()
}
stats_df = pd.DataFrame(stats)
print("\n--- Normalized Score Statistics ---\n", stats_df)

# Subject with highest variability
most_variable_subject = stats_df["StdDev"].idxmax()
print("\nMost variable subject (highest StdDev):", most_variable_subject)

# Top 3 students by average marks
top_students = df.nlargest(3, "AverageMarks")[["StudentID", "Name", "AverageMarks"]]
print("\nTop 3 Students:\n", top_students)

# Subject-wise average performance (pivot table)
pivot_avg = df[["Name", "Math", "Physics", "Chemistry"]].melt(id_vars=["Name"],
                                                               var_name="Subject",
                                                               value_name="Marks")
pivot_table = pivot_avg.pivot_table(index="Subject", values="Marks", aggfunc="mean")
print("\n--- Subject-wise Average Performance ---\n", pivot_table)



--- Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       7 non-null      object 
 1   Math       6 non-null      float64
 2   Physics    5 non-null      float64
 3   Chemistry  5 non-null      float64
dtypes: float64(3), object(1)
memory usage: 356.0+ bytes
None

--- Dataset Description ---
           Math    Physics  Chemistry
count   6.00000   5.000000   5.000000
mean   80.00000  85.000000  88.400000
std     3.63318   4.636809   1.516575
min    75.00000  79.000000  87.000000
25%    78.00000  84.000000  88.000000
50%    80.00000  85.000000  88.000000
75%    82.00000  85.000000  88.000000
max    85.00000  92.000000  91.000000

Missing values per subject:
Math         1
Physics      2
Chemistry    2
dtype: int64

Duplicate Entries:
     Name  Math  Physics  Chemistry
0  Alice  78.0     85.0        NaN
1    Bob  82.0      NaN    

/tmp/ipython-input-2519278286.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Math"].fillna(df["Math"].median(), inplace=True)
/tmp/ipython-input-2519278286.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 



# Exercise 3.b: Movie Ratings Aggregation and Review Analysis
#### Problem Statement:
Analyze movie ratings and review text data to find average ratings, detect duplicates, and extract insights using string operations.

#### Objective Scenario:
A streaming platform wants to analyze user reviews. The dataset contains user IDs, movie titles, ratings, and review comments. Some reviews are duplicated and some comments contain extra whitespace or punctuation noise.

#### Dataset:

Stored in movie_reviews.csv with the columns:

Sample Data:

UserID,MovieTitle,Rating,ReviewText

U001,Inception,4.5,"Amazing movie!"

U002,Inception,4.0,"  brilliant  "

U001,Inception,4.5,"Amazing movie!"

U003,Interstellar,5.0,"Mind-blowing!"

U004,Interstellar,, "Outstanding visuals."

U005,Inception,4.2,"Good plot but confusing"

#### Tasks to be Performed:

🔹 Cleaning and Preprocessing

•	Remove duplicate reviews based on UserID and MovieTitle.
•	Handle missing ratings by filling with average rating per movie.
•	Strip and lowercase all review text using vectorized string operations.

🔹 Aggregation

•	Group by MovieTitle and calculate mean, median, and count of ratings.
•	Identify the top-rated movie.

🔹 Text Analysis

•	Count how many times the word “amazing” appears in all reviews.
•	Filter reviews where rating is above 4.0 and review contains the word “plot”.


In [3]:
import pandas as pd

# ------------------------------------
# 🔹 1. Load Data
# ------------------------------------
df = pd.read_csv("/content/movie_reviews.csv")

print("\n--- Original Data ---")
print(df)

# ------------------------------------
# 🔹 Cleaning & Preprocessing
# ------------------------------------
# Remove duplicates based on UserID & MovieTitle (keep first occurrence)
df = df.drop_duplicates(subset=["UserID", "MovieTitle"], keep="first")

# Fill missing ratings with movie-wise average
df["Rating"] = df.groupby("MovieTitle")["Rating"].transform(
    lambda x: x.fillna(x.mean())
)

# Strip extra whitespace & lowercase review text
df["ReviewText"] = df["ReviewText"].str.strip().str.lower()

print("\n--- Cleaned Data ---")
print(df)

# ------------------------------------
# 🔹 Aggregation
# ------------------------------------
movie_stats = df.groupby("MovieTitle")["Rating"].agg(["mean", "median", "count"])
top_rated_movie = movie_stats["mean"].idxmax()

print("\n--- Movie Statistics ---")
print(movie_stats)
print("\nTop Rated Movie:", top_rated_movie)

# ------------------------------------
# 🔹 Text Analysis
# ------------------------------------
# Count occurrences of "amazing" in all reviews
amazing_count = df["ReviewText"].str.count(r"\bamazing\b").sum()

# Filter reviews with rating > 4.0 and containing "plot"
plot_reviews = df[
    (df["Rating"] > 4.0) &
    (df["ReviewText"].str.contains("plot", case=False, na=False))
]

print("\nOccurrences of 'amazing':", amazing_count)
print("\nReviews with rating > 4.0 and containing 'plot':")
print(plot_reviews)



--- Original Data ---
  UserID    MovieTitle  Rating               ReviewText
0   U001     Inception     4.5           Amazing movie!
1   U002     Inception     4.0               brilliant 
2   U001     Inception     4.5           Amazing movie!
3   U003  Interstellar     5.0            Mind-blowing!
4   U004  Interstellar     NaN     Outstanding visuals.
5   U005     Inception     4.2  Good plot but confusing

--- Cleaned Data ---
  UserID    MovieTitle  Rating               ReviewText
0   U001     Inception     4.5           amazing movie!
1   U002     Inception     4.0                brilliant
3   U003  Interstellar     5.0            mind-blowing!
4   U004  Interstellar     5.0     outstanding visuals.
5   U005     Inception     4.2  good plot but confusing

--- Movie Statistics ---
                  mean  median  count
MovieTitle                           
Inception     4.233333     4.2      3
Interstellar  5.000000     5.0      2

Top Rated Movie: Interstellar

Occurrences of 'a